In [ ]:
torch.cuda.empty_cache()

In [ ]:
import glob
class GANImageDataset(Dataset):
    def __init__(self, weather, town, model, transform=None, target_transform=None):
        self.dir = './Data/'+weather+'/'+town+'/'+model+'/test_latest/images'
        self.transform = transform
        self.target_transform = target_transform
        self.real = glob.glob(self.dir+'/*real.png')


    def __len__(self):
        return len(list(self.real))

    def __getitem__(self, idx): 
        img_path = self.real[idx]
        image = read_image(img_path)
        label_name_split = self.real[idx].split('_')
        label_name = label_name_split[0]+'_'+label_name_split[1]+'_fake.png'
        label = read_image( label_name)
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
import torchvision.transforms.functional as TF
import random
import os
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import carla
from utils import AE_initalize_weights
from utils import generate_semantic_im
from utils import replace
from utils import GANImageDataset

from torch.utils.tensorboard import SummaryWriter

from torch.utils.data import DataLoader, Subset 
from torch.utils.data import ConcatDataset
from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision.utils import save_image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class PerceptionNet(nn.Module):

    def __init__(self):
        super(PerceptionNet,self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=4, stride=2, padding=1)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.conv4 = nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        
        self.conv5 = nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        
        self.conv6a = nn.Conv2d(512, 64, kernel_size=4, stride=1)
        self.conv6b = nn.Conv2d(512, 64, kernel_size=4, stride=1)
        
        self.conv7 = torch.nn.ConvTranspose2d(64,512, kernel_size =4, stride=1)
        self.bn6 = nn.BatchNorm2d(512)
        
        self.conv8 = torch.nn.ConvTranspose2d(512,256, kernel_size =4, stride=2, padding=1)
        self.bn7 = nn.BatchNorm2d(256)
        
        self.conv9 = torch.nn.ConvTranspose2d(256,128, kernel_size =4, stride=2, padding=1)
        self.bn8 = nn.BatchNorm2d(128)
        
        self.conv10 = torch.nn.ConvTranspose2d(128,64, kernel_size =4, stride=2, padding=1)
        self.bn9 = nn.BatchNorm2d(64)
        
        self.conv11 = torch.nn.ConvTranspose2d(64,32, kernel_size =4, stride=2, padding=1)
        self.bn10 = nn.BatchNorm2d(32)
        
        self.conv12 = torch.nn.ConvTranspose2d(32,13, kernel_size =4, stride=2,padding=1)
        
            
    def encode(self, x):
        x = F.leaky_relu(self.conv1(x),negative_slope=0.02)
        x = F.leaky_relu(self.bn2(self.conv2(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn3(self.conv3(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn4(self.conv4(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn5(self.conv5(x)),negative_slope=0.02)
        return self.conv6a(x)

    
    def decode(self, x):
        x = F.leaky_relu(self.bn6(self.conv7(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn7(self.conv8(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn8(self.conv9(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn9(self.conv10(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn10(self.conv11(x)),negative_slope=0.02)
        return torch.sigmoid(self.conv12(x))

    
    def forward(self, x):
        x = x.to(device, dtype=torch.float32)
        latent = self.encode(x)
        out = self.decode(latent)
        return out, latent

In [ ]:
master = PerceptionNet()
servant = PerceptionNet()

master.to(device)
master.load_state_dict(torch.load('./AE_params/model_46.final'))
master.eval()

servant.to(device)
servant.apply(AE_initalize_weights)
servant.train()


In [6]:
# Change to dry-wet for Hard Rain and final wetcyclegan4 for best model
train_data = GANImageDataset('dry-cloudysunset','Town04','cloudysunsetcyclegan')
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

In [ ]:
optimizer = torch.optim.Adam(servant.parameters(), lr=0.0003)
loss_fn = nn.MSELoss()

In [ ]:
def train(epoch):
    global step
    servant.train()
    writer = SummaryWriter()
    train_loss = 0
    
    for batch_idx, data in enumerate(train_loader):
        org_domain = data[0]
        new_domain = data[1]
        batch_size = new_domain.shape[0]
        with torch.no_grad():
            _,latent_true = master(org_domain)
        
        optimizer.zero_grad()
        _,latent_hat = servant(new_domain)
        loss = loss_fn(latent_hat,latent_true)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_size * batch_idx, len(train_loader.dataset),
            100. * batch_idx / len(train_loader),
            loss.item() / len(data)))

        writer.add_scalar("AE Loss", loss.item() / len(data[0]),step)
        step += 1

    avg_loss = train_loss / len(train_loader.dataset)
    print('====> Epoch: {} Average loss: {:.8f}'.format(
          epoch, avg_loss))
    writer.flush()
    return avg_loss
    
for epoch in range(1, 10 + 1):
    step = 0
    smallest_loss = 1000
    avg_loss = train(epoch)
    if avg_loss < smallest_loss:
        torch.save(servant.state_dict(), './AE_params/dry-cloudysunset_2.best')
    #test(epoch)
    torch.save(servant.state_dict(), './AE_params/dry-cloudysunset_2.final')

In [ ]:
num = random.randint(1,18000)
data = train_data.__getitem__(num)
imgs = []
org = data[0]
new = data[1]
servant.eval()

_,latent = master(data[0].reshape(1,3,128,128))
_,new_lat = servant(data[1].reshape(1,3,128,128))
imgs.append(Image.fromarray(org.numpy().transpose(1,2,0)))
imgs.append(Image.fromarray(new.numpy().transpose(1,2,0)))

sem = master.decode(new_lat).detach().cpu().argmax(dim=1)

imgs.append(generate_semantic_im(org,master))
imgs.append(replace(sem.numpy().reshape(1,128,128).transpose(1,2,0)))
fix, axs = plt.subplots(ncols=len(imgs), squeeze=False)
for i, img in enumerate(imgs):
    axs[0, i].imshow(np.asarray(img))
    axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])